# Do Data Show Construction Labor Market Shortages?

Construction company executives are increasingly complaining that they are not able to find qualified workers and need government help. While a tight labor market can result in a situation where not enough workers are available, the overall labor force participation rate and employment to population ratio both suggest that there are still unemployed people willing to work. Looking at the data, this is particularly true in the construction industry. With lots of potential workers still on the sidelines, construction employers can easily grow their workforce to meet demand. The obvious way to do this is by offering wages that pull in skilled workers and/or providing training to get new workers up to speed. It seems, however, that executives are trying very different techniques--getting everyone else to pay to train new construction workers and importing low-wage labor from other countries. 

#### Points: 

1) Unemployed per job opening nearly 3x the total for all jobs.<br>
2) Average weekly hours are basically flat and currently lower than 2015 level.<br>
3) Real wage growth is below total private.<br>
4) Construction is seasonal (average hours worked by month) and very subject to swings in the economy (construction layoffs vs total).<br>
5) Construction will be slowed by Fed rate hikes (higher borrowing costs for builders).

#### Policies that increase the supply of low-wage workers:

1) Training programs that put too much emphasis on the skills that executives demand rather than the skills which most benefit workers. <br>
2) Visas to bring in low-wage workers from overseas in cases where there are plenty of domestic workers with the appropriate skills. 

#### Technical preliminaries

Python 2.7

In [2]:
import requests
import json
import pandas as pd
import config # file which contains API keys
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.grid_objs import Column, Grid

plotly.tools.set_credentials_file(username='bdew', api_key=config.plotly_key)

#### Request data from BLS

In [3]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}   # Request json data
sd = {
    'TOT_AHE': 'CES0500000008',  # Average hourly earnings
    'CON_AHE': 'CES2000000008',
    'CON_JOU': 'JTU23000000JOL', # Job openings (level) NSA
    'TOT_JOU': 'JTU00000000JOL',
    'CON_UN': 'LNU03032231',     # Unemployed persons NSA
    'TOT_UN': 'LNU03000000',
    'CON_HRS': 'CES2000000007',  # Weekly hours
    'CPI-U': 'CUSR0000SA0'      # CPI for all urban consumers
}

In [4]:
data = json.dumps({"seriesid":sd.values(), "startyear":'2014', "endyear":'2017'})
p = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()

#### Build pandas dataframe from results

In [5]:
# Empty dictionary. Each entry will be one series
d = {}

# Loop through the series and convert to datetime indexed float values
for series in p['Results']['series']:
    s = series['seriesID']  # Short name, 's', for series ID
    d[s] = pd.DataFrame(series['data'])
    d[s]['date'] = pd.to_datetime(d[s]['period'] + ' ' + d[s]['year'])
    d[s] = d[s].set_index('date')['value'].astype(float)

# Combine the individual dictionary entries into one dataframe
df = pd.concat(d, axis=1)

# Show last five rows
df.tail(14)

,CES0500000008,CES2000000007,CES2000000008,CUSR0000SA0,JTU00000000JOL,JTU23000000JOL,LNU03000000,LNU03032231
date,,,,,,,,
2016-05-01,21.48,39.6,25.86,239.362,5591.0,187.0,7207.0,461.0
2016-06-01,21.53,39.7,26.05,239.842,5467.0,171.0,8144.0,417.0
2016-07-01,21.59,39.7,26.12,239.898,6254.0,238.0,8267.0,410.0
2016-08-01,21.62,39.4,26.07,240.389,5581.0,184.0,7996.0,454.0
2016-09-01,21.68,39.6,26.21,241.006,5676.0,237.0,7658.0,474.0
2016-10-01,21.72,39.6,26.28,241.694,5853.0,196.0,7447.0,512.0
2016-11-01,21.74,39.8,26.24,242.199,5379.0,178.0,7066.0,517.0
2016-12-01,21.80,39.2,26.23,242.821,5116.0,140.0,7170.0,670.0
2017-01-01,21.83,39.3,26.33,244.158,5557.0,142.0,8149.0,859.0


#### Basic calculations for real wages and unemployed per job opening

In [6]:
# Real wages for construction (CON) and total private
df['CON_RW_12CH'] = df[sd['CON_AHE']].multiply(
    df[sd['CPI-U']][0]).divide(
    df[sd['CPI-U']]).pct_change(12).multiply(100)
df['TOT_RW_12CH'] = df[sd['TOT_AHE']].multiply(
    df[sd['CPI-U']][0]).divide(
    df[sd['CPI-U']]).pct_change(12).multiply(100)
# Unemployed per job opening
df['CON_UNOP'] = df[sd['CON_UN']].divide(df[sd['CON_JOU']])
df['TOT_UNOP'] = df[sd['TOT_UN']].divide(df[sd['TOT_JOU']])

#### Save data to a csv file

In [7]:
df.to_csv('construction_data.csv')

## Review the results

#### Unemployed per job opening

Down overall, up for construction over the past year

In [8]:
# One year ago
df[['CON_UNOP', 'TOT_UNOP']].dropna().iloc[-13]

CON_UNOP    2.465241
TOT_UNOP    1.289036
Name: 2016-05-01 00:00:00, dtype: float64

In [9]:
# Latest Available
df[['CON_UNOP', 'TOT_UNOP']].dropna().iloc[-1]

CON_UNOP    3.259740
TOT_UNOP    1.160106
Name: 2017-05-01 00:00:00, dtype: float64

In [11]:
# Unemployed per job opening

trace1 = {
  "x": [2.465241, 1.289036], 
  "y": ["Construction", "All sectors"], 
  "error_y": {"visible": False}, 
  "marker": {
    "color": "rgb(0, 128, 255)"
  }, 
  "name": "May 2016", 
  "hoverinfo": "text", 
  "text": ["Construction, May 2016<br>Ratio: 2.5<br>Openings: 187,000<br>Unemployed: 461,000", "All sectors total, May 2016<br>Ratio: 1.3<br>Openings: 5,591,000<br>Unemployed: 7,207,000"], 
  "orientation": "h", 
  "type": "bar", 
}
trace2 = {
  "x": [3.259740, 1.160106], 
  "y": ["Construction", "All sectors"], 
  "marker": {
    "color": "rgb(0, 0, 153)" 
  }, 
  "name": "May 2017", 
  "hoverinfo": "text", 
  "text": ["Construction, May 2017<br>Ratio: 3.3<br>Openings: 154,000<br>Unemployed: 430,000", "All sectors total, May 2017<br>Ratio: 1.2<br>Openings: 5,665,000<br>Unemployed: 7,250,000"], 
  "orientation": "h", 
  "type": "bar", 
}
data = Data([trace1, trace2])
layout = {
  "annotations": [
    {
      "x": 0.5, 
      "y": 0.815789473684, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "May 2016", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.5, 
      "y": 1.19473684211, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "May 2017", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 3.4901276391, 
      "y": 0.226315789474, 
      "font": {"size": 16}, 
      "showarrow": False, 
      "text": "<b>3.3</b>", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 2.68310112796, 
      "y": -0.194736842105, 
      "font": {"size": 16}, 
      "showarrow": False, 
      "text": "<b>2.5</b>", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 1.37299315532, 
      "y": 1.19473684211, 
      "font": {"size": 16}, 
      "showarrow": False, 
      "text": "<b>1.2</b>", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 1.47780179313, 
      "y": 0.794736842105, 
      "font": {"size": 16}, 
      "showarrow": False, 
      "text": "<b>1.3</b>", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": -0.348868131868, 
      "y": 1.53834586466, 
      "align": "left", 
      "font": {"size": 16}, 
      "showarrow": False, 
      "text": "<b>The construction sector has more unemployed&lt;br&gt;workers per job opening</b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.296835164835, 
      "y": 1.22706766918, 
      "align": "left", 
      "font": {"size": 13}, 
      "showarrow": False, 
      "text": "<i>Ratio of unemployed workers per job opening</i>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.35, 
      "y": -0.55, 
      "align": "left", 
      "font": {"size": 11}, 
      "showarrow": False, 
      "text": "bluecollarjobs.us&lt;br&gt;Source: Bureau of Labor Statistics, Labor Force Statistics and JOLTS.<br>Series: Construction: <a href=\"https://data.bls.gov/timeseries/LNU03032231\">LNU03032231 </a>; <a href=\"https://data.bls.gov/timeseries/JTU23000000JOL\">JTU23000000JOL</a>;<br>All sectors: <a href=\"https://data.bls.gov/timeseries/LNU03000000\">LNU03000000 </a>; <a href=\"https://data.bls.gov/timeseries/JTU00000000JOL\">JTU00000000JOL</a>", 
      "xref": "paper", 
      "yref": "paper"
    }
  ], 
  "autosize": False, 
  "bargap": 0.2, 
  "bargroupgap": 0.15, 
  "barmode": "group", 
  "boxgap": 0.3, 
  "boxgroupgap": 0.3, 
  "boxmode": "overlay", 
  "font": {
    "color": "#444", 
    "family": "Open Sans", 
    "size": 16
  }, 
  "height": 300, 
  "hidesources": False, 
  "hovermode": "closest", 
  "margin": {
    "r": 10, 
    "t": 80, 
    "autoexpand": True, 
    "b": 80, 
    "l": 110, 
    "pad": 0
  }, 
  "paper_bgcolor": "rgb(255, 255, 255)", 
  "plot_bgcolor": "rgb(255, 255, 255)", 
  "separators": ".,", 
  "showlegend": False, 
  "smith": False, 
  "title": "", 
  "width": 400, 
  "xaxis": {
    "anchor": "y", 
    "autorange": True, 
    "domain": [0, 1], 
    "dtick": 5, 
    "exponentformat": "B", 
    "mirror": False, 
    "nticks": 0, 
    "overlaying": False, 
    "position": 0, 
    "range": [0, 3.68073724651], 
    "rangemode": "normal", 
    "showgrid": False, 
    "showline": False, 
    "showticklabels": False, 
    "tick0": 0, 
    "ticklen": 5, 
    "tickmode": "auto", 
    "ticks": "", 
    "tickwidth": 1, 
    "title": "", 
    "type": "linear", 
    "zeroline": False
  }, 
  "yaxis": {
    "anchor": "x", 
    "autorange": True, 
    "domain": [0, 1], 
    "dtick": 1, 
    "mirror": False, 
    "nticks": 0, 
    "overlaying": False, 
    "position": 0, 
    "range": [-0.5, 1.5], 
    "rangemode": "normal",  
    "showgrid": False, 
    "showline": False, 
    "showticklabels": True, 
    "tick0": 0, 
    "tickangle": "auto", 
    "tickcolor": "#444", 
    "tickfont": {
      "color": "", 
      "family": "", 
      "size": 14
    }, 
    "ticklen": 5, 
    "tickmode": "auto", 
    "ticks": "", 
    "ticksuffix": " ", 
    "tickwidth": 1, 
    "title": "", 
    "type": "category", 
    "zeroline": False, 
  }
}
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='Construction_Labor_Market_fig1_test')

#### Hours worked

Flat over the past three years

In [12]:
df[sd['CON_HRS']].iloc[-37:]#.plot()

date
2014-06-01    39.6
2014-07-01    39.9
2014-08-01    39.8
2014-09-01    39.7
2014-10-01    39.3
2014-11-01    39.7
2014-12-01    39.8
2015-01-01    39.7
2015-02-01    39.7
2015-03-01    39.4
2015-04-01    39.3
2015-05-01    39.4
2015-06-01    39.8
2015-07-01    39.5
2015-08-01    39.8
2015-09-01    39.6
2015-10-01    40.0
2015-11-01    39.7
2015-12-01    40.2
2016-01-01    39.8
2016-02-01    39.6
2016-03-01    39.4
2016-04-01    39.5
2016-05-01    39.6
2016-06-01    39.7
2016-07-01    39.7
2016-08-01    39.4
2016-09-01    39.6
2016-10-01    39.6
2016-11-01    39.8
2016-12-01    39.2
2017-01-01    39.3
2017-02-01    39.8
2017-03-01    39.3
2017-04-01    39.9
2017-05-01    39.9
2017-06-01    39.7
Freq: MS, Name: CES2000000007, dtype: float64

In [13]:
awe_series = pd.DataFrame(df[sd['CON_HRS']].iloc[-37:])
awe_series['new'] = ''
awe_series['new'].iloc[0] = awe_series['CES2000000007'][0]
awe_series['new'].iloc[-1] = awe_series['CES2000000007'][-1]
for index, row in awe_series.iterrows():
    awe_series.loc[index,'text'] = 'Average Weekly Hours:<br>{}: {}'.format(
        pd.to_datetime(index).strftime('%B %Y'),
        awe_series.loc[index,'CES2000000007'])
    awe_series.loc[index,'new_text'] = '{}<br>{}'.format(
        pd.to_datetime(index).strftime('%B %Y'),
        awe_series.loc[index,'new'])

C:\Anaconda2\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [14]:
# Average Weekly Hours Plot

trace1 = {
  "x": awe_series.index, 
  "y": awe_series['CES2000000007'], 
  "mode": "lines", 
  "hoverinfo": "text", 
  "name": "Average Weekly Hours, Construction", 
  "text": awe_series['text'], 
  "type": "scatter"
}
trace2 = {
  "x": awe_series.index,
  "y": awe_series['new'], 
  "hoverinfo": "none", 
  "marker": {"size": 10,  "color": "rgb(31, 119, 180)"}, 
  "mode": "markers+text", 
  "name": "", 
  "text": awe_series['new_text'],  
  "textfont": {"size": 14}, 
  "textposition": "top center", 
  "type": "scatter", 
}
data = Data([trace1, trace2])
layout = {
  "annotations": [
    {
      "x": -0.052, 
      "y": -0.41, 
      "align": "left", 
      "font": {
        "family": "Open Sans, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "bluecollarjobs.us<br>Source: Bureau of Labor Statistics, labor force statistics from the &lt;br&gt;current population survey, series code <a href=\"https://data.bls.gov/timeseries/CES2000000007\">CES2000000007</a>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.0559324782545, 
      "y": 1.2576984127, 
      "align": "left", 
      "font": {
        "family": "Open Sans", 
        "size": 16
      }, 
      "showarrow": False, 
      "text": "<b>Construction worker hours are essentially flat<br>over the past three years</b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.0382352941176, 
      "y": 1.0, 
      "align": "left", 
      "font": {"size": 13}, 
      "showarrow": False, 
      "text": "<i>Average weekly hours worked, production and nonsupervisory<br>employees, seasonally adjusted.</i>", 
      "xref": "paper", 
      "yref": "paper"
    }
  ], 
  "autosize": False, 
  "height": 300, 
  "hovermode": "closest", 
  "legend": {
    "x": 0.616334319709, 
    "y": 0.306177606176, 
    "font": {"size": 12}, 
    "orientation": "v", 
    "traceorder": "normal"
  }, 
  "margin": {
    "r": 30, 
    "t": 50, 
    "b": 70, 
    "l": 30
  }, 
  "paper_bgcolor": "rgb(255, 255, 255)", 
  "plot_bgcolor": "rgb(255, 255, 255)", 
  "showlegend": False, 
  "title": "", 
  "titlefont": {"size": 16}, 
  "width": 400, 
  "xaxis": {
    "anchor": "y", 
    "autorange": False, 
    "domain": [0, 1], 
    "fixedrange": True, 
    "nticks": 5, 
    "range": ["2014-01-02", "2017-12-01"], 
    "showgrid": False, 
    "side": "bottom", 
    "tickfont": {
      "color": "rgb(127, 127, 127)", 
      "family": "Open Sans"
    }, 
    "ticks": "inside", 
    "ticksuffix": "", 
    "title": "", 
    "type": "date", 
    "zeroline": False
  }, 
  "yaxis": {
    "autorange": False, 
    "fixedrange": True, 
    "range": [33, 50], 
    "showgrid": False, 
    "showticklabels": False, 
    "tick0": 0, 
    "tickmode": "linear", 
    "ticks": "", 
    "ticksuffix": "%", 
    "title": "", 
    "type": "linear"
  }
}
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='Construction_Labor_Market_fig2_test')

#### Real wage growth

About the same in construction as in all sectors -- very low!

In [12]:
df[['TOT_RW_12CH', 'CON_RW_12CH']].dropna().iloc[-1:]

,TOT_RW_12CH,CON_RW_12CH
date,,
2017-06-01,0.665306,0.608771


In [17]:
# Real wage growth

trace1 = {
  "x": ["0.61", "0.67"], 
  "y": ["Construction", "Total Private"], 
  "marker": {
    "color": "rgb(31, 119, 180)"
  }, 
  "name": "Real wage growth", 
  "hoverinfo": "text", 
  "text": ["Construction, June 2017<br>Real wage growth: 0.61%<br>Average hourly earnings: $26.24", "Total private, June 2017<br>Real wage growth: 0.67%<br>Average hourly earnings: $22.03"], 
  "opacity": 1, 
  "orientation": "h", 
  "type": "bar", 
}
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": -0.448, 
      "y": 1.53571428571, 
      "font": {"size": 16}, 
      "showarrow": False, 
      "align": "left", 
      "text": "<b>Real hourly earnings growth is very low</b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.417, 
      "y": 1.34285714286, 
      "font": {"size": 13}, 
      "showarrow": False, 
      "align": "left", 
      "text": "<i>June 2017, annual growth rate of average hourly earnings,&lt;br&gt;production and nonsupervisory employees</i>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": 0.451851851852, 
      "y": 0.842857142857, 
      "showarrow": False, 
      "text": "<b>0.67%</b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": 0.425925925926, 
      "y": 0.135714285714, 
      "showarrow": False, 
      "text": "<b>0.61%</b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.456, 
      "y": -0.41, 
      "font": {"size": 12}, 
      "showarrow": False, 
      "align": "left", 
      "text": "bluecollarjobs.us&lt;br&gt;Source: Bureau of Labor Statistics. Adjusted for inflation using <a href=\"https://data.bls.gov/timeseries/CUSR0000SA0\">CPI-U</a>.<br>Series: Construction: <a href=\"https://data.bls.gov/timeseries/CES2000000008\">CES2000000008</a>; Total private: <a href=\"https://data.bls.gov/timeseries/CES0500000008\">CES0500000008</a>", 
      "xref": "paper", 
      "yref": "paper"
    }
  ], 
  "autosize": False, 
  "bargap": 0.35, 
  "bargroupgap": 0.15, 
  "barmode": "group", 
  "boxgap": 0.3, 
  "boxgroupgap": 0.3, 
  "font": {
    "color": "#444", 
    "family": "Open Sans", 
    "size": 16
  }, 
  "height": 300, 
  "hidesources": False, 
  "margin": {
    "r": 0, 
    "t": 80, 
    "b": 80, 
    "l": 130, 
    "pad": 0
  }, 
  "paper_bgcolor": "rgb(255, 255, 255)", 
  "plot_bgcolor": "rgb(255, 255, 255)", 
  "title": "", 
  "width": 400, 
  "xaxis": {
    "anchor": "y", 
    "autorange": False, 
    "domain": [0, 1], 
    "dtick": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "linecolor": "#444", 
    "linewidth": 1, 
    "mirror": False, 
    "nticks": 0, 
    "overlaying": False, 
    "position": 0, 
    "range": [0, 2], 
    "rangemode": "normal", 
    "showexponent": "all", 
    "showgrid": True, 
    "showline": False, 
    "showticklabels": False, 
    "tick0": 0, 
    "tickangle": "auto", 
    "tickcolor": "#444", 
    "tickfont": {
      "color": "", 
      "family": "", 
      "size": 0
    }, 
    "ticklen": 5, 
    "tickmode": "auto", 
    "ticks": "", 
    "tickwidth": 1, 
    "title": "", 
    "titlefont": {
      "color": "", 
      "family": "", 
      "size": 0
    }, 
    "type": "linear", 
    "zeroline": True, 
    "zerolinecolor": "rgb(255, 255, 255)", 
    "zerolinewidth": 1
  }, 
  "yaxis": {
    "anchor": "x", 
    "autorange": True, 
    "domain": [0, 1], 
    "dtick": 1, 
    "mirror": False, 
    "nticks": 0, 
    "overlaying": False, 
    "position": 0, 
    "range": [-0.5, 1.5], 
    "rangemode": "normal", 
    "showexponent": "all", 
    "showgrid": False, 
    "showline": False, 
    "showticklabels": True, 
    "tick0": 0, 
    "ticklen": 5, 
    "tickmode": "auto", 
    "ticks": "", 
    "ticksuffix": " ", 
    "tickwidth": 1, 
    "title": "", 
    "type": "category", 
  }
}
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='Construction_Labor_Market_fig3_test')